In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
## https://covid.ourworldindata.org/data/ecdc/full_data.csv
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN


In [2]:
c.tail(3)

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
17010,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17011,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17012,NaN,International,2020-03-10,696,-9,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-12,France,139834.0,0.0,26808.0,0.0,0.0,0.0
0,2020-05-12,China,84093.0,0.0,4659.0,0.0,0.0,0.0
0,2020-05-12,Italy,220643.0,0.0,30907.0,0.0,0.0,0.0
0,2020-05-12,Spain,228283.0,0.0,26869.0,0.0,0.0,0.0
0,2020-05-12,United States,1369131.0,0.0,81422.0,0.0,0.0,0.0
0,2020-05-12,World,4223326.0,0.0,289384.0,0.0,0.0,0.0
0,2020-05-12,United Kingdom,227330.0,0.0,32261.0,0.0,0.0,0.0
0,2020-05-12,Germany,170916.0,0.0,7586.0,0.0,0.0,0.0
0,2020-05-12,Iran,110773.0,0.0,6744.0,0.0,0.0,0.0
0,2020-05-12,Turkey,141125.0,0.0,3881.0,0.0,0.0,0.0


In [5]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [7]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-13,France,139834.0,140227.0,26808.0,26991.0,-393.0,-183.0
0,2020-05-13,China,84093.0,84018.0,4659.0,4637.0,75.0,22.0
0,2020-05-13,Italy,220643.0,221216.0,30907.0,30911.0,-573.0,-4.0
0,2020-05-12,Spain,228283.0,228030.0,26869.0,26920.0,253.0,-51.0
0,2020-05-13,United States,1369131.0,1369964.0,81422.0,82387.0,-833.0,-965.0
0,2020-05-13,World,4223326.0,4223047.0,289384.0,291519.0,279.0,-2135.0
0,2020-05-13,United Kingdom,227330.0,226463.0,32261.0,32692.0,867.0,-431.0
0,2020-05-13,Germany,170916.0,171306.0,7586.0,7634.0,-390.0,-48.0
0,2020-05-13,Iran,110773.0,110767.0,6744.0,6733.0,6.0,11.0
0,2020-05-13,Turkey,141125.0,141475.0,3881.0,3894.0,-350.0,-13.0


In [8]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")